# 讀取歷史資料

In [1]:
import sqlite3

In [2]:
import pandas as pd

In [3]:
with sqlite3.connect('C:/Users/poduo/Anaconda3_JupyterNB/20171217Python爬蟲實戰/BOT_currency/BOT_JPY.sqlite') as db:
    df = pd.read_sql_query('select * from BOT_JPY', con = db)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 394 entries, 0 to 393
Data columns (total 7 columns):
index      394 non-null int64
掛牌日期       394 non-null object
幣別         394 non-null object
現金-本行買入    394 non-null float64
現金-本行賣出    394 non-null float64
即期-本行買入    394 non-null float64
即期-本行賣出    394 non-null float64
dtypes: float64(4), int64(1), object(2)
memory usage: 21.6+ KB


In [5]:
df.head(5)

,index,掛牌日期,幣別,現金-本行買入,現金-本行賣出,即期-本行買入,即期-本行賣出
0,0,2018-01-25 00:00:00,日圓 (JPY),0.2583,0.2693,0.2647,0.2687
1,1,2018-01-25 00:00:00,日圓 (JPY),0.2584,0.2694,0.2648,0.2688
2,2,2018-01-25 00:00:00,日圓 (JPY),0.2583,0.2693,0.2647,0.2687
3,3,2018-01-24 00:00:00,日圓 (JPY),0.2573,0.2683,0.2637,0.2677
4,4,2018-01-23 00:00:00,日圓 (JPY),0.2553,0.2663,0.2617,0.2657


# 抓取最新資料

In [6]:
o_data = pd.read_html('http://rate.bot.com.tw/xrt/quote/l6m/JPY')
jpy_his = o_data[0].iloc[:,0:6]
jpy_his.columns = ['掛牌日期', '幣別', '現金-本行買入', '現金-本行賣出', '即期-本行買入', '即期-本行賣出']
from datetime import datetime
jpy_his = jpy_his.set_index(jpy_his['掛牌日期'])
del jpy_his['掛牌日期']
jpy_his.index = pd.to_datetime(jpy_his.index, format='%Y/%m/%d')

### 取出最近1筆日期

In [7]:
today_jpy = jpy_his.head(1)

In [23]:
today_jpy

,幣別,現金-本行買入,現金-本行賣出,即期-本行買入,即期-本行賣出
掛牌日期,,,,,
2018-01-25,日圓 (JPY),0.2581,0.2691,0.2645,0.2685


### 寫入資料庫
最新資料寫入 dataFrame

整個 dataFrame 再寫入 SQLite

In [36]:
with sqlite3.connect('BOT_JPY.sqlite') as db:
    today_jpy.to_sql('BOT_JPY', con= db, if_exists = 'append')

In [37]:
with sqlite3.connect('C:/Users/poduo/Anaconda3_JupyterNB/20171217Python爬蟲實戰/BOT_currency/BOT_JPY.sqlite') as db:
    df = pd.read_sql_query('select * from BOT_JPY', con = db)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 131 entries, 0 to 130
Data columns (total 7 columns):
index      130 non-null float64
掛牌日期       131 non-null object
幣別         131 non-null object
現金-本行買入    131 non-null float64
現金-本行賣出    131 non-null float64
即期-本行買入    131 non-null float64
即期-本行賣出    131 non-null float64
dtypes: float64(5), object(2)
memory usage: 7.2+ KB


In [38]:
df = df.sort_values(by= ['掛牌日期'], ascending=False).reset_index(drop = True)
df.head(5)

,index,掛牌日期,幣別,現金-本行買入,現金-本行賣出,即期-本行買入,即期-本行賣出
0,0.0,2018-01-25 00:00:00,日圓 (JPY),0.2581,0.2691,0.2645,0.2685
1,NaN,2018-01-25 00:00:00,日圓 (JPY),0.2581,0.2691,0.2645,0.2685
2,1.0,2018-01-24 00:00:00,日圓 (JPY),0.2573,0.2683,0.2637,0.2677
3,2.0,2018-01-23 00:00:00,日圓 (JPY),0.2553,0.2663,0.2617,0.2657
4,3.0,2018-01-22 00:00:00,日圓 (JPY),0.2565,0.2675,0.2629,0.2669


In [39]:
df = df.drop_duplicates(['掛牌日期'], keep = 'last').reset_index(drop = True)
df.head(5)

,index,掛牌日期,幣別,現金-本行買入,現金-本行賣出,即期-本行買入,即期-本行賣出
0,NaN,2018-01-25 00:00:00,日圓 (JPY),0.2581,0.2691,0.2645,0.2685
1,1.0,2018-01-24 00:00:00,日圓 (JPY),0.2573,0.2683,0.2637,0.2677
2,2.0,2018-01-23 00:00:00,日圓 (JPY),0.2553,0.2663,0.2617,0.2657
3,3.0,2018-01-22 00:00:00,日圓 (JPY),0.2565,0.2675,0.2629,0.2669
4,4.0,2018-01-19 00:00:00,日圓 (JPY),0.2573,0.2683,0.2637,0.2677


In [40]:
del df['index']
df.head(5)

,掛牌日期,幣別,現金-本行買入,現金-本行賣出,即期-本行買入,即期-本行賣出
0,2018-01-25 00:00:00,日圓 (JPY),0.2581,0.2691,0.2645,0.2685
1,2018-01-24 00:00:00,日圓 (JPY),0.2573,0.2683,0.2637,0.2677
2,2018-01-23 00:00:00,日圓 (JPY),0.2553,0.2663,0.2617,0.2657
3,2018-01-22 00:00:00,日圓 (JPY),0.2565,0.2675,0.2629,0.2669
4,2018-01-19 00:00:00,日圓 (JPY),0.2573,0.2683,0.2637,0.2677


In [41]:
with sqlite3.connect('BOT_JPY.sqlite') as db:
    df.to_sql('BOT_JPY', con= db, if_exists = 'replace')

## 畫最新折線圖

In [42]:
from datetime import datetime
df = df.set_index(df['掛牌日期'])
del df['掛牌日期']
df.index = pd.to_datetime(df.index, format='%Y/%m/%d')

In [43]:
import plotly
import plotly.plotly as py
import plotly.graph_objs as go

plotly.tools.set_credentials_file(username='s842563', api_key='gDEK6deJwLf0LwhD9BzC')

trace_bank_sale = go.Scatter(x = df.index,
                y=df['即期-本行賣出'],
                name = "即期匯率-本行賣出",
                line = dict(color = '#17BECF'),
                opacity = 0.8)

trace_bank_buy = go.Scatter(x = df.index,
                y=df['即期-本行買入'],
                name = "即期匯率-本行買入",
                line = dict(color = 'red'),
                opacity = 0.8)

data = [trace_bank_sale, trace_bank_buy]

layout = dict(
    title = df['幣別'][0],
    xaxis = dict(range = [df.index[-1], df.index[0]])
             )

fig = dict(data=data, layout=layout)

py.iplot(fig, filename = df['幣別'][0])